# 🎓 STUDENT_GEN - AI Learning Assistant
Complete learning agent with notes, quizzes, DSA helper, coding helper, and gamification.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from xhtml2pdf import pisa
from typing import List, Optional
from enum import Enum
from datetime import datetime
import os, json, re, markdown

load_dotenv()
if os.environ.get('GEMINI_API_KEY'):
    print('✅ API KEY LOADED')
else:
    raise ValueError('❌ API KEY NOT FOUND')

model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.7)

In [ ]:
# ==================== PYDANTIC MODELS ====================
class DifficultyLevel(str, Enum):
    EASY = 'easy'
    MEDIUM = 'medium'
    HARD = 'hard'

class Option(BaseModel):
    label: str = Field(description='Option label (A, B, C, D)')
    text: str = Field(description='Option text')
    is_correct: bool = Field(description='Is correct answer')

class Question(BaseModel):
    question_number: int
    question_text: str
    options: List[Option]
    correct_answer: str
    explanation: str
    difficulty: DifficultyLevel
    topic_tag: str

class Quiz(BaseModel):
    title: str
    topic: str
    total_questions: int
    time_limit_minutes: int
    questions: List[Question]
    passing_score: int

class Note(BaseModel):
    title: str
    topic: str
    summary: str
    key_points: List[str]
    detailed_content: str
    examples: List[str]
    practice_questions: List[str]

class YouTubeResource(BaseModel):
    video_title: str
    channel_name: str
    duration: str
    url: str

class Book(BaseModel):
    title: str
    author: str
    focus_area: str

class Project(BaseModel):
    name: str
    description: str
    deliverables: List[str]

class Week(BaseModel):
    week_number: int
    theme: str
    hours_per_day: int
    topics: List[str]
    youtube_resources: List[YouTubeResource]
    books: List[Book]
    goals: List[str]
    projects: List[Project]

class Roadmap(BaseModel):
    title: str
    topic: str
    total_weeks: int
    difficulty_level: str
    weekly_commitment_hours: int
    prerequisites: List[str]
    weeks: List[Week]
    skills_acquired: List[str]
    next_steps: List[str]

class DSAProblem(BaseModel):
    problem_name: str
    difficulty: DifficultyLevel
    problem_statement: str
    examples: List[str]
    constraints: List[str]
    hints: List[str]
    solution_approach: str
    code_solution: str
    time_complexity: str
    space_complexity: str

class AlgorithmExplanation(BaseModel):
    name: str
    category: str
    description: str
    steps: List[str]
    pseudocode: str
    time_complexity: str
    space_complexity: str
    use_cases: List[str]
    code_example: str

class CodeReview(BaseModel):
    original_code: str
    issues_found: List[str]
    suggestions: List[str]
    improved_code: str
    explanation: str

class CodeDebug(BaseModel):
    original_code: str
    error_message: str
    bug_explanation: str
    fixed_code: str
    prevention_tips: List[str]

class InterviewQuestion(BaseModel):
    question: str
    category: str
    difficulty: DifficultyLevel
    sample_answer: str
    follow_up_questions: List[str]
    tips: List[str]

class StepByStepExplanation(BaseModel):
    topic: str
    introduction: str
    prerequisites: List[str]
    steps: List[str]
    examples: List[str]
    common_mistakes: List[str]
    summary: str
    next_topics: List[str]

class WeaknessAnalysis(BaseModel):
    weak_topics: List[str]
    score_percentage: int
    recommendations: List[str]
    practice_focus: List[str]
    resources: List[str]

print('✅ Models Loaded')

In [ ]:
# ==================== SYSTEM PROMPTS ====================
NOTES_PROMPT = '''You are an expert educator. Create comprehensive study notes on the given topic.
Include: clear summary, key points, detailed explanations, examples, and practice questions.
Make it student-friendly with proper structure.'''

QUIZ_PROMPT = '''You are an expert quiz designer. Create a well-structured quiz.
Include: MCQ questions with 4 options each, varying difficulty (30% easy, 50% medium, 20% hard).
Each question must have clear explanation of the correct answer.'''

ROADMAP_PROMPT = '''You are a curriculum designer. Create a detailed week-by-week learning roadmap.
Include: topics, YouTube resources with real URLs, books, weekly goals, and hands-on projects.
Progress from beginner to advanced concepts.'''

DSA_EXPLAIN_PROMPT = '''You are a DSA expert. Explain the algorithm step-by-step.
Include: description, visual steps, pseudocode, complexity analysis, use cases, and working code.'''

DSA_PROBLEM_PROMPT = '''You are a competitive programming expert. Create a DSA practice problem.
Include: problem statement, examples, constraints, hints, solution approach, code, and complexity.'''

CODE_DEBUG_PROMPT = '''You are a debugging expert. Analyze the code and error.
Find the bug, explain why it occurs, provide fixed code, and give prevention tips.'''

CODE_REVIEW_PROMPT = '''You are a senior developer. Review this code.
Find issues, suggest improvements, provide better code, and explain changes.'''

CODE_EXPLAIN_PROMPT = '''You are a coding tutor. Explain this code line by line.
Break down each part, explain logic, and highlight important concepts.'''

INTERVIEW_PROMPT = '''You are an interview coach. Generate interview questions for the role/topic.
Include: question, category, difficulty, sample answer, follow-ups, and tips.'''

STEP_BY_STEP_PROMPT = '''You are a patient tutor. Explain this topic step-by-step for beginners.
Include: intro, prerequisites, numbered steps, examples, common mistakes, summary, next topics.'''

WEAKNESS_PROMPT = '''You are a learning analyst. Analyze quiz results and identify weak areas.
Provide: weak topics, score analysis, recommendations, practice focus, and resources.'''

print('✅ Prompts Loaded')

In [ ]:
# ==================== UTILITY FUNCTIONS ====================
def save_pdf(content: str, filepath: str, title: str):
    html = markdown.markdown(content, extensions=['tables', 'fenced_code'])
    styled = f'''<!DOCTYPE html><html><head><meta charset="UTF-8"><style>
    @page {{ size: A4; margin: 1cm; }}
    body {{ font-family: Arial; padding: 20px; line-height: 1.6; font-size: 11px; }}
    h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; font-size: 20px; }}
    h2 {{ color: #34495e; font-size: 16px; }}
    h3 {{ color: #7f8c8d; font-size: 13px; }}
    table {{ border-collapse: collapse; width: 100%; font-size: 9px; }}
    th, td {{ border: 1px solid #ddd; padding: 6px; }}
    th {{ background: #3498db; color: white; }}
    code {{ background: #f4f4f4; padding: 2px 5px; }}
    pre {{ background: #2d2d2d; color: #f8f8f2; padding: 10px; }}
    </style></head><body>{html}</body></html>'''
    with open(filepath, 'w+b') as f:
        pisa.CreatePDF(styled, dest=f)
    print(f'📄 PDF saved: {filepath}')

def log_activity(activity_type: str, details: str, xp: int):
    os.makedirs('logs', exist_ok=True)
    date_str = datetime.now().strftime('%Y-%m-%d')
    log_file = f'logs/{date_str}_activity.jsonl'
    entry = {'timestamp': datetime.now().isoformat(), 'type': activity_type, 'details': details, 'xp': xp}
    with open(log_file, 'a') as f:
        f.write(json.dumps(entry) + '\n')
    update_progress(xp)
    print(f'📝 Logged: +{xp} XP for {activity_type}')

def update_progress(xp_earned: int):
    progress_file = 'progress.json'
    if os.path.exists(progress_file):
        with open(progress_file) as f:
            data = json.load(f)
    else:
        data = {'xp': 0, 'level': 1, 'activities': 0, 'streak': 0, 'achievements': []}
    data['xp'] += xp_earned
    data['activities'] += 1
    data['level'] = get_level(data['xp'])
    with open(progress_file, 'w') as f:
        json.dump(data, f, indent=2)

def get_level(xp: int) -> int:
    if xp >= 1000: return 5
    if xp >= 600: return 4
    if xp >= 300: return 3
    if xp >= 100: return 2
    return 1

def show_stats():
    if not os.path.exists('progress.json'):
        print('📊 No progress yet. Start learning!')
        return
    with open('progress.json') as f:
        d = json.load(f)
    levels = {1: 'Novice', 2: 'Learner', 3: 'Scholar', 4: 'Expert', 5: 'Master'}
    print(f'''\n🎮 === YOUR STATS ===
⭐ Level: {d['level']} ({levels[d['level']]})
💎 XP: {d['xp']}
📚 Activities: {d['activities']}
🔥 Streak: {d['streak']} days\n''')

def sanitize_name(name: str) -> str:
    return re.sub(r'[^\w\s-]', '', name).replace(' ', '_').lower()

print('✅ Utilities Loaded')

In [ ]:
# ==================== NOTES GENERATOR ====================
def generate_notes(topic: str) -> Note:
    os.makedirs('notes', exist_ok=True)
    prompt = ChatPromptTemplate.from_messages([('system', NOTES_PROMPT), ('human', 'Create notes on: {topic}')])
    chain = prompt | model.with_structured_output(Note)
    result = chain.invoke({'topic': topic})
    
    md = f'# {result.title}\n\n## Summary\n{result.summary}\n\n'
    md += '## Key Points\n' + '\n'.join([f'- {p}' for p in result.key_points]) + '\n\n'
    md += f'## Detailed Content\n{result.detailed_content}\n\n'
    md += '## Examples\n' + '\n'.join([f'{i+1}. {e}' for i, e in enumerate(result.examples)]) + '\n\n'
    md += '## Practice Questions\n' + '\n'.join([f'{i+1}. {q}' for i, q in enumerate(result.practice_questions)])
    
    name = sanitize_name(topic)
    with open(f'notes/{name}_notes.md', 'w', encoding='utf-8') as f:
        f.write(md)
    save_pdf(md, f'notes/{name}_notes.pdf', result.title)
    log_activity('notes', topic, 10)
    return result

# Test: generate_notes('Machine Learning Basics')

In [ ]:
# ==================== ROADMAP GENERATOR ====================
def generate_roadmap(topic: str) -> Roadmap:
    os.makedirs('roadmaps', exist_ok=True)
    prompt = ChatPromptTemplate.from_messages([('system', ROADMAP_PROMPT), ('human', 'Create roadmap for: {topic}')])
    chain = prompt | model.with_structured_output(Roadmap)
    result = chain.invoke({'topic': topic})
    
    md = f'# {result.title}\n\n**Duration:** {result.total_weeks} Weeks | **Level:** {result.difficulty_level}\n\n'
    md += '## Prerequisites\n' + '\n'.join([f'- {p}' for p in result.prerequisites]) + '\n\n---\n\n'
    
    for w in result.weeks:
        md += f'## Week {w.week_number}: {w.theme}\n**Effort:** {w.hours_per_day} hrs/day\n\n'
        md += '### Topics\n' + '\n'.join([f'- {t}' for t in w.topics]) + '\n\n'
        md += '### Videos\n| Video | Channel | Duration |\n|-------|---------|----------|\n'
        for y in w.youtube_resources:
            md += f'| {y.video_title} | {y.channel_name} | {y.duration} |\n'
        md += '\n### Goals\n' + '\n'.join([f'- [ ] {g}' for g in w.goals]) + '\n\n---\n\n'
    
    md += '## Skills Acquired\n' + '\n'.join([f'- {s}' for s in result.skills_acquired])
    
    name = sanitize_name(topic)
    with open(f'roadmaps/{name}_roadmap.md', 'w', encoding='utf-8') as f:
        f.write(md)
    save_pdf(md, f'roadmaps/{name}_roadmap.pdf', result.title)
    log_activity('roadmap', topic, 25)
    return result

# Test: generate_roadmap('Full Stack Web Development')

In [ ]:
# ==================== QUIZ GENERATOR ====================
def generate_quiz(topic: str) -> Quiz:
    os.makedirs('quizzes', exist_ok=True)
    prompt = ChatPromptTemplate.from_messages([('system', QUIZ_PROMPT), ('human', 'Create quiz on: {topic}')])
    chain = prompt | model.with_structured_output(Quiz)
    result = chain.invoke({'topic': topic})
    
    md = f'# {result.title}\n\n**Topic:** {result.topic} | **Questions:** {result.total_questions} | **Time:** {result.time_limit_minutes} mins\n\n---\n\n'
    for q in result.questions:
        md += f'### Q{q.question_number}. {q.question_text}\n*Difficulty: {q.difficulty.value}*\n\n'
        for opt in q.options:
            md += f'- **{opt.label})** {opt.text}\n'
        md += f'\n**Answer:** {q.correct_answer}\n\n**Explanation:** {q.explanation}\n\n---\n\n'
    
    name = sanitize_name(topic)
    with open(f'quizzes/{name}_quiz.md', 'w', encoding='utf-8') as f:
        f.write(md)
    save_pdf(md, f'quizzes/{name}_quiz.pdf', result.title)
    log_activity('quiz', topic, 15)
    return result

def analyze_weakness(correct: int, total: int, wrong_topics: List[str]) -> WeaknessAnalysis:
    prompt = ChatPromptTemplate.from_messages([('system', WEAKNESS_PROMPT), 
        ('human', 'Score: {correct}/{total}. Wrong topics: {topics}. Analyze weaknesses.')])
    chain = prompt | model.with_structured_output(WeaknessAnalysis)
    return chain.invoke({'correct': correct, 'total': total, 'topics': ', '.join(wrong_topics)})

# Test: generate_quiz('Python Programming')

In [ ]:
# ==================== DSA HELPER ====================
def explain_algorithm(algo_name: str) -> AlgorithmExplanation:
    prompt = ChatPromptTemplate.from_messages([('system', DSA_EXPLAIN_PROMPT), ('human', 'Explain: {algo}')])
    chain = prompt | model.with_structured_output(AlgorithmExplanation)
    result = chain.invoke({'algo': algo_name})
    
    print(f'''\n🔢 {result.name} ({result.category})
📝 {result.description}\n
📋 Steps:\n''' + '\n'.join([f'{i+1}. {s}' for i,s in enumerate(result.steps)]) + f'''
\n⏱️ Time: {result.time_complexity} | Space: {result.space_complexity}
\n💻 Code:\n{result.code_example}''')
    log_activity('dsa_explain', algo_name, 10)
    return result

def generate_dsa_problem(topic: str, difficulty: str = 'medium') -> DSAProblem:
    prompt = ChatPromptTemplate.from_messages([('system', DSA_PROBLEM_PROMPT), 
        ('human', 'Create {difficulty} problem on: {topic}')])
    chain = prompt | model.with_structured_output(DSAProblem)
    result = chain.invoke({'topic': topic, 'difficulty': difficulty})
    
    print(f'''\n🎯 {result.problem_name} [{result.difficulty.value}]
\n📜 Problem:\n{result.problem_statement}
\n📊 Examples:\n''' + '\n'.join(result.examples) + f'''
\n💡 Hints:\n''' + '\n'.join([f'{i+1}. {h}' for i,h in enumerate(result.hints)]))
    log_activity('dsa_problem', topic, 15)
    return result

def show_dsa_solution(problem: DSAProblem):
    print(f'''\n✅ Solution Approach:\n{problem.solution_approach}
\n💻 Code:\n{problem.code_solution}
\n⏱️ Time: {problem.time_complexity} | Space: {problem.space_complexity}''')

# Test: explain_algorithm('Binary Search')
# Test: generate_dsa_problem('Arrays', 'easy')

In [ ]:
# ==================== CODING HELPER ====================
def debug_code(code: str, error: str) -> CodeDebug:
    prompt = ChatPromptTemplate.from_messages([('system', CODE_DEBUG_PROMPT), 
        ('human', 'Code:\n```\n{code}\n```\nError: {error}')])
    chain = prompt | model.with_structured_output(CodeDebug)
    result = chain.invoke({'code': code, 'error': error})
    
    print(f'''\n🐛 Bug Found:\n{result.bug_explanation}
\n✅ Fixed Code:\n{result.fixed_code}
\n💡 Prevention Tips:\n''' + '\n'.join([f'- {t}' for t in result.prevention_tips]))
    log_activity('debug', 'code_fix', 10)
    return result

def review_code(code: str) -> CodeReview:
    prompt = ChatPromptTemplate.from_messages([('system', CODE_REVIEW_PROMPT), 
        ('human', 'Review this code:\n```\n{code}\n```')])
    chain = prompt | model.with_structured_output(CodeReview)
    result = chain.invoke({'code': code})
    
    print(f'''\n🔍 Issues Found:\n''' + '\n'.join([f'- {i}' for i in result.issues_found]) + 
          f'''\n\n💡 Suggestions:\n''' + '\n'.join([f'- {s}' for s in result.suggestions]) +
          f'''\n\n✨ Improved Code:\n{result.improved_code}''')
    log_activity('review', 'code_review', 10)
    return result

def explain_code(code: str):
    prompt = ChatPromptTemplate.from_messages([('system', CODE_EXPLAIN_PROMPT), 
        ('human', 'Explain line by line:\n```\n{code}\n```')])
    response = model.invoke(prompt.format_messages(code=code))
    print(f'\n📖 Code Explanation:\n{response.content}')
    log_activity('explain', 'code_explain', 5)

# Test: debug_code('def add(a, b):\n  return a + c', 'NameError: c is not defined')

In [ ]:
# ==================== INTERVIEW PREP ====================
class InterviewSet(BaseModel):
    role: str
    questions: List[InterviewQuestion]

def generate_interview_questions(role: str, topic: str, count: int = 5) -> InterviewSet:
    prompt = ChatPromptTemplate.from_messages([('system', INTERVIEW_PROMPT), 
        ('human', 'Generate {count} interview questions for {role} role on {topic}')])
    chain = prompt | model.with_structured_output(InterviewSet)
    result = chain.invoke({'role': role, 'topic': topic, 'count': count})
    
    for q in result.questions:
        print(f'''\n❓ {q.question}
📂 Category: {q.category} | Difficulty: {q.difficulty.value}
\n✅ Sample Answer:\n{q.sample_answer}
\n🔄 Follow-ups:\n''' + '\n'.join([f'- {f}' for f in q.follow_up_questions]))
        print('---')
    log_activity('interview', f'{role}_{topic}', 20)
    return result

# Test: generate_interview_questions('Software Engineer', 'System Design', 3)

In [ ]:
# ==================== STEP-BY-STEP LEARNING ====================
def explain_step_by_step(topic: str) -> StepByStepExplanation:
    prompt = ChatPromptTemplate.from_messages([('system', STEP_BY_STEP_PROMPT), 
        ('human', 'Explain step-by-step: {topic}')])
    chain = prompt | model.with_structured_output(StepByStepExplanation)
    result = chain.invoke({'topic': topic})
    
    print(f'''\n📚 {result.topic}
\n📖 Introduction:\n{result.introduction}
\n📋 Prerequisites:\n''' + '\n'.join([f'- {p}' for p in result.prerequisites]) + 
          f'''\n\n🚶 Steps:\n''' + '\n'.join([f'{i+1}. {s}' for i,s in enumerate(result.steps)]) +
          f'''\n\n💡 Examples:\n''' + '\n'.join(result.examples) +
          f'''\n\n⚠️ Common Mistakes:\n''' + '\n'.join([f'- {m}' for m in result.common_mistakes]) +
          f'''\n\n📝 Summary:\n{result.summary}
\n➡️ Next Topics:\n''' + '\n'.join([f'- {t}' for t in result.next_topics]))
    log_activity('learn', topic, 15)
    return result

# Test: explain_step_by_step('How to use Git and GitHub')

In [ ]:
# ==================== TEXT SUMMARIZER ====================
class Summary(BaseModel):
    original_length: int
    summary: str
    key_points: List[str]
    main_topics: List[str]

def summarize_text(text: str) -> Summary:
    prompt = ChatPromptTemplate.from_messages([('system', 'Summarize the text concisely. Extract key points and main topics.'), 
        ('human', 'Summarize:\n{text}')])
    chain = prompt | model.with_structured_output(Summary)
    result = chain.invoke({'text': text})
    
    print(f'''\n📝 Summary ({result.original_length} chars → {len(result.summary)} chars):
{result.summary}
\n🔑 Key Points:\n''' + '\n'.join([f'- {p}' for p in result.key_points]))
    log_activity('summarize', 'text', 5)
    return result

# Test: summarize_text('Your long text here...')

In [ ]:
# ==================== INTERACTIVE MENU ====================
def main_menu():
    show_stats()
    print('''\n🎓 STUDENT_GEN - AI Learning Assistant
═══════════════════════════════════════
1. 📝 Generate Notes (topic → PDF)
2. 🗺️ Generate Roadmap (topic → PDF)
3. 📊 Generate Quiz (topic → PDF)
4. 🔢 DSA Helper (explain algorithm)
5. 🎯 DSA Problem (practice problem)
6. 🐛 Debug Code (fix errors)
7. 🔍 Review Code (improvements)
8. 📖 Explain Code (line by line)
9. 🎤 Interview Prep (mock questions)
10. 🚶 Step-by-Step Learning
11. 📄 Summarize Text
12. 📊 View Stats
0. Exit
═══════════════════════════════════════''')

while True:
    main_menu()
    choice = input('\nChoose option: ')
    
    if choice == '0': print('👋 Goodbye!'); break
    elif choice == '1': generate_notes(input('Topic: '))
    elif choice == '2': generate_roadmap(input('Topic: '))
    elif choice == '3': generate_quiz(input('Topic: '))
    elif choice == '4': explain_algorithm(input('Algorithm name: '))
    elif choice == '5': generate_dsa_problem(input('Topic: '), input('Difficulty (easy/medium/hard): '))
    elif choice == '6': debug_code(input('Paste code: '), input('Error message: '))
    elif choice == '7': review_code(input('Paste code: '))
    elif choice == '8': explain_code(input('Paste code: '))
    elif choice == '9': generate_interview_questions(input('Role: '), input('Topic: '))
    elif choice == '10': explain_step_by_step(input('Topic: '))
    elif choice == '11': summarize_text(input('Paste text: '))
    elif choice == '12': show_stats()
    else: print('❌ Invalid option')